In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # Install required libs
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install albumentations==0.4.6

In [3]:
# !sudo rm -rf /content/seg_recyclables
# !git clone https://github.com/finani/seg_recyclables.git

# import sys
# sys.path.append('/content/seg_recyclables')

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print('')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print('')

import os
import torch
print('Pytorch version: {}'.format(torch.__version__))
print('Is GPU available: {}'.format(torch.cuda.is_available()))
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
  print('The number of GPUs available: {}'.format(torch.cuda.device_count())) # Tesla P100-PCIE-16GB
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print('CPU count: {}'.format(os.cpu_count()))  # 8

cuda = torch.version.cuda
cudnn = torch.backends.cudnn.version()
cudnn_major = cudnn // 1000
cudnn = cudnn % 1000
cudnn_minor = cudnn // 100
cudnn_patch = cudnn % 100
print('Cuda version: {}'.format(cuda)) # 11.1
print('Cudnn version: {}.{}.{}'.format(cudnn_major, cudnn_minor, cudnn_patch)) # 8.0.5


Tue Dec 28 16:43:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P5    18W /  N/A |    855MiB /  7982MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Train 12 Segmentation
Classes =  ['Background', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

## Configures

In [5]:
import os
import sys

import torch
import numpy as np

import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler

import Utils
from ModelManager import ModelManager
from DataManager import DataManager, CustomAugmentation
from TrainManager import TrainManager
from InferManager import InferManager
from LossManager import DiceLoss
from LearningRateManager import CustomCosineAnnealingWarmUpRestarts

Utils.fix_random_seed(random_seed=21)

project_dir = '/home/weebee/recyclables/baseline'
dataset_dir = os.path.join(project_dir, 'input')
save_dir = os.path.join(project_dir, 'saved/tm_test')
if not os.path.isdir(dataset_dir):
    sys.exit('check dataset path!!')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)


## Train

In [6]:
target_classes = Utils.get_classes()

config_dict = {
    'project_name': 'test',
    'run_name': '[TM] All_CE_e5',
    'network': 'DeepLabV3Plus',
    'encoder': 'resnet101',
    'encoder_weights': 'imagenet',
    'target_classes': target_classes,
    'activation': None,
    'multi_gpu': False,
    'num_epochs': 5,
    'batch_size': 5,
    'learning_rate_0': 1e-4,
    'number_worker': 4,
    'val_every': 1,
    'note': 'test train'
}

# Make Model
model_manager = ModelManager()
model = model_manager.make_deeplabv3plus_model(
    encoder=config_dict['encoder'],
    encoder_weights=config_dict['encoder_weights'],
    class_number=len(target_classes), # 12
    activation=config_dict['activation'],
    multi_gpu=config_dict['multi_gpu']
)

# Load Dataset
data_manager = DataManager(dataset_path=dataset_dir)
data_manager.assignDataLoaders(
    batch_size=config_dict['batch_size'],
    shuffle=True,
    number_worker=config_dict['number_worker'],
    drop_last=False,
    transform=CustomAugmentation.to_tensor_transform() # should be list
)

criterion = smp.utils.losses.CrossEntropyLoss()
# criterion = smp.utils.losses.BCEWithLogitsLoss()
# criterion = tgm.losses.DiceLoss()
# criterion = DiceLoss()
# criterion = smp.utils.losses.JaccardLoss()

optimizer = torch.optim.Adam(
    [dict(params=model.parameters(),
            lr=config_dict['learning_rate_0']
            ),
        ])

lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=0.01, steps_per_epoch=10, epochs=epochs, anneal_strategy='cos'
# )
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     optimizer, T_0=1, T_mult=2, eta_min=5e-5,
# )
# lr_scheduler = CustomCosineAnnealingWarmUpRestarts(
#     optimizer, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5
# )

# Run Train
train_manager = TrainManager()
train_manager.run_train(
    model=model,
    config_dict=config_dict,
    data_loader=data_manager.train_data_loader,
    val_loader=data_manager.val_data_loader,
    criterion=criterion,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    save_dir=save_dir,
    file_name='best_model_1.pt',
)

loading annotations into memory...
Done (t=2.26s)
creating index...
index created!
loading annotations into memory...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Done (t=0.73s)
creating index...
index created!

Start training..
num_epochs: 5
save_dir: /home/weebee/recyclables/baseline/saved/tm_test
file_name: best_model_1.pt
val_every: 1



wandb: Currently logged in as: inhwan-wee (use `wandb login --relogin` to force relogin)


Epoch: 0/5, Time: 2021-12-28 16:44:07, lr: 0.0001: 100%|██████████| 524/524 [04:40<00:00,  1.87it/s, loss=0.62, mean_loss=0.711]


...

Epoch: 4/5, Time: 2021-12-28 17:04:31, lr: 0.0001: 100%|██████████| 524/524 [04:43<00:00,  1.85it/s, loss=0.709, mean_loss=0.239]



Start validation #5


Epoch: 5: 100%|██████████| 131/131 [00:23<00:00,  5.65it/s, mIoU_all=0.387, mIoU_batch=0.324]



Validation #5  Average Loss: 0.3806, mIoU_all: 0.3873


DeepLabV3Plus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia